## Data load

In [1]:
from data_text import *
import numpy as np
from params import *

In [2]:
params = Params()
params.DATA_PATH='../data/target_seven_120/debug/'
dataset = DataText(params, 'train')
len(dataset)

[DEBUG] load data : train_nlp_trans.npy train_label.npy


6058

In [3]:
from torch.utils.data import DataLoader

data_loader_train = DataLoader(dataset=dataset,
                          batch_size=2,
                          shuffle=True,
                          num_workers=8)
len(data_loader_train)

3029

In [4]:
for t, s, l in data_loader_train:
#     print(t, s, l)
    break    

In [5]:
# dataset.load_glove()

In [6]:
t

tensor([[3500, 1359,  933,  954, 1513,  270,  665, 1611,  667, 3610],
        [1045, 2864, 1430,    0,    0,    0,    0,    0,    0,    0]])

## model - final

In [1]:
import torch
from torch.nn.utils.rnn import pack_padded_sequence, pad_sequence, pad_packed_sequence
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [11]:
from model_text import *
from data_text import *

params = Params()
params.DATA_PATH='../data/target_seven_120/debug/'
dataset = DataText(params, 'dev')
print(len(dataset), dataset.dic_size)

params.BATCH_SIZE = 2
params.DIC_SIZE = dataset.dic_size
params.ENCDOER_SIZE = 10
params.NUM_LAYER = 2
params.USE_GLOVE = int(1)
params.HIDDEN_DIM = 5
params.EMBED_DIM = 20
params.LR = 1e-3
params.DR = 0.0
params.DEVICE = device
params.EMBEDDING_FINETUNE = False


model = ModelText(dic_size     = params.DIC_SIZE,
                  embed_dim    = params.EMBED_DIM,
                  hidden_dim   = params.HIDDEN_DIM,
                  num_layers   = params.NUM_LAYER,
                  num_category = params.N_CATEGORY,
                  dr           = params.DR,
                  use_glove    = params.USE_GLOVE,
                  glove_embedding    = dataset.load_glove(),
                  embedding_finetune = params.EMBEDDING_FINETUNE,
                  use_attention = 1,
                  memory_dim = 256
                  )

[DEBUG] load data : dev_nlp_trans.npy dev_label.npy
404 3747
[INFO] load Glove pretrained model
[DEBUG][object created]  ModelText
[DEBUG] create attention memory, dim:  5
[DEBUG] Embedding size: 300
[DEBUG][launch-text] create embedding
[DEBUG] Glove finetuning: False
[DEBUG][launch-text] create text_encoder (GRU):
[DEBUG][launch-text] create output projection layer


In [12]:
model.to(params.DEVICE)
model.memory = model.memory.to(params.DEVICE)
# model.to('cuda')
# model.to('cpu')
params.DEVICE

device(type='cuda')

In [13]:
model.memory.shape

torch.Size([256])

In [14]:
from torch.utils.data import DataLoader
data_loader = DataLoader(dataset=dataset,
                          batch_size=params.BATCH_SIZE,
                          shuffle=True,
                          num_workers=8)
len(data_loader)

for t, m, l in data_loader:
    break
t = t.to(device)
m = m.to(device)
l = l.to(device)

In [15]:
final_output = model(t, m)
final_output, m

(tensor([[-0.1675,  0.0661, -0.3546, -0.3705,  0.2415],
         [ 0.0343,  0.0649, -0.1586, -0.3070,  0.3610]], device='cuda:0',
        grad_fn=<SumBackward1>),
 tensor([[1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
         [1., 1., 1., 1., 1., 0., 0., 0., 0., 0.]], device='cuda:0'))

## model debug

In [1]:
import torch
from torch.nn.utils.rnn import pack_padded_sequence, pad_sequence, pad_packed_sequence
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device
from model_attention_pytorch import luong_attention

In [2]:
from model_text import *
from data_text import *

params = Params()
params.DATA_PATH='../data/target_seven_120/debug/'
dataset = DataText(params, 'dev')
print(len(dataset), dataset.dic_size)

params.BATCH_SIZE = 2
params.DIC_SIZE = dataset.dic_size
params.ENCDOER_SIZE = 10
params.NUM_LAYER = 3
params.USE_GLOVE = int(1)
params.HIDDEN_DIM = 5
params.EMBED_DIM = 20
params.LR = 1e-3
params.DR = 0.0
params.DEVICE = device
params.EMBEDDING_FINETUNE = False


model = ModelText(dic_size     = params.DIC_SIZE,
                  embed_dim    = params.EMBED_DIM,
                  hidden_dim   = params.HIDDEN_DIM,
                  num_layers   = params.NUM_LAYER,
                  num_category = params.N_CATEGORY,
                  dr           = params.DR,
                  use_glove    = params.USE_GLOVE,
                  glove_embedding    = dataset.load_glove(),
                  embedding_finetune = params.EMBEDDING_FINETUNE,
                  use_attention = 1,
                  memory_dim = 128
                  )

model.to(params.DEVICE)
model.memory = model.memory.to(params.DEVICE)

[DEBUG] load data : dev_nlp_trans.npy dev_label.npy
404 3747
[INFO] load Glove pretrained model
[DEBUG][object created]  ModelText
[DEBUG] create attention memory, dim:  5
[DEBUG] Embedding size: 300
[DEBUG][launch-text] create embedding
[DEBUG] Glove finetuning: False
[DEBUG][launch-text] create text_encoder (GRU):
[DEBUG][launch-text] create output projection layer


In [3]:
from torch.utils.data import DataLoader
data_loader = DataLoader(dataset=dataset,
                          batch_size=params.BATCH_SIZE,
                          shuffle=True,
                          num_workers=8)
len(data_loader)

for t, m, l in data_loader:
    break
t = t.to(device)
m = m.to(device)
l = l.to(device)
t, m

(tensor([[1992, 3610,    0,    0,    0,    0,    0,    0,    0,    0],
         [1782,  621, 2707, 1726, 2661,    0,    0,    0,    0,    0]],
        device='cuda:0'), tensor([[1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 1., 1., 1., 1., 0., 0., 0., 0., 0.]], device='cuda:0'))

In [4]:
embed = model.fn_embed(t)
seqN  = torch.sum(m, dim=-1)

In [5]:
packed_outputs, h_n = model.fn_encoder(pack_padded_sequence(input=embed,
                                                    lengths=seqN,
                                                    batch_first=True,
                                                    enforce_sorted=False
                                                   )
                               )

In [6]:
outputs, output_lengths = pad_packed_sequence(packed_outputs,
                                              batch_first=True,
                                              padding_value=0.0,
                                              total_length=10
                                             )
outputs.shape

torch.Size([2, 10, 5])

In [9]:
query = model.linear(model.memory).unsqueeze(0).unsqueeze(-1)
key   = outputs
query.shape, key.shape

(torch.Size([1, 5, 1]), torch.Size([2, 10, 5]))

In [10]:
weighted_sum, norm_b_sim = luong_attention(key, query, m)

## train

In [1]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [4]:
from train_text import *
from model_text import *
from data_text import *

params = Params()
params.DATA_PATH='../data/target_seven_120/debug/'
dataset = DataText(params, 'dev')
print(len(dataset), dataset.dic_size)

params.BATCH_SIZE = 64
params.DIC_SIZE = dataset.dic_size
params.ENCDOER_SIZE = 10
params.NUM_LAYER = 1
params.USE_GLOVE = 0
params.HIDDEN_DIM = 5
params.EMBED_DIM = 20
params.LR = 1e-3
params.DR = 0.0
params.ATTENTION = 0
params.DEVICE = device

    
model = ModelText(dic_size     = params.DIC_SIZE,
                  embed_dim    = params.EMBED_DIM,
                  hidden_dim   = params.HIDDEN_DIM,
                  num_layers   = params.NUM_LAYER,
                  num_category = params.N_CATEGORY,
                  use_glove    = params.USE_GLOVE,
                  dr           = params.DR,
                  use_attention = 1,
                  memory_dim   = params.ATTENTION_MEM_DIM
                  )
model.to(params.DEVICE)
if params.ATTENTION:
    model.memory = model.memory.to(params.DEVICE)
# model.to('cpu')

[DEBUG] load data : dev_nlp_trans.npy dev_label.npy
404 3747
[DEBUG][object created]  ModelText
[DEBUG] create attention memory, dim:  5
[DEBUG] Embedding size: 20
[DEBUG][launch-text] create embedding
[DEBUG] Glove finetuning: True
[DEBUG][launch-text] create text_encoder (GRU):
[DEBUG][launch-text] create output projection layer


In [6]:
model.memory.shape

torch.Size([256])

In [7]:
optimizer = torch.optim.Adam(model.parameters(), lr=params.LR)
criterion = nn.CrossEntropyLoss()

In [8]:
dataset_dev = DataText(params, 'test')
len(dataset)
data_loader_dev = DataLoader(dataset=dataset_dev,
                               batch_size=params.BATCH_SIZE,
                               shuffle=False,
                               num_workers=4)
for b_trans, b_seq_mask, b_label in data_loader_dev:
    break

[DEBUG] load data : test_nlp_trans.npy test_label.npy


In [9]:
b_loss_train = train_step(params,
                          model,
                          b_trans,
                          b_seq_mask,
                          b_label,
                          optimizer,
                          criterion)

RuntimeError: Expected object of device type cuda but got device type cpu for argument #1 'self' in call to _th_mm

In [10]:
train_model(params, model, dataset, dataset, dataset, 3)

excepetion occurs in train step:  Expected object of device type cuda but got device type cpu for argument #1 'self' in call to _th_mm
excepetion occurs in train step:  Expected object of device type cuda but got device type cpu for argument #1 'self' in call to _th_mm
excepetion occurs in evaluate step :  Expected object of device type cuda but got device type cpu for argument #1 'self' in call to _th_mm


UnboundLocalError: local variable 'b_loss' referenced before assignment

In [11]:
from evaluation import evaluate

In [12]:
dev_ce, dev_WA, dev_UA = evaluate(params=params,
                                  model=model, 
                                  data_loader=data_loader_dev
                                 )

In [13]:
with torch.no_grad():
    criterion = nn.CrossEntropyLoss(reduction='sum')

    list_loss = []
    list_pred = []
    list_label = []

    for b_trans, b_seqN, b_label in data_loader_dev:

        b_trans = b_trans.to(params.DEVICE)
        b_seqN  = b_seqN.to(params.DEVICE)
        b_label = b_label.to(params.DEVICE)

        try:
            b_pred = model(b_trans, b_seqN)
            b_loss = criterion(b_pred, b_label)

        except Exception as e:
            print ('excepetion occurs in evaluate step : ', e)

        # batch loss
        list_loss.append( b_loss )

        # batch prediction
        list_pred.append( torch.argmax(input=b_pred, dim=-1) )
        list_label.append( b_label )

    # flatten
    list_pred  = torch.cat(list_pred)
    list_label = torch.cat(list_label)

In [14]:
model.memory

AttributeError: 'ModelText' object has no attribute 'memory'

In [ ]:
with torch.autograd.profiler.profile(use_cuda=True) as prof:
     model(b_trans, b_seqN)

print(prof)